In [2]:
# Packages
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET

## 1. Data Preprocessing

### 1.1 FileSetup

In [3]:
os.chdir('C:/Users/ConquerV/Documents/2022Summer/EconResearch_TAML/TextMining')
path = os.getcwd()
print(path)
xmlfile = 'journal_sample.xml'

C:\Users\ConquerV\Documents\2022Summer\EconResearch_TAML\TextMining


### Web-based xml (Kept for future use)

In [86]:
# def loadRSS():
    # url of rss feed
#     url = '.xml'
  
#     # creating HTTP response object from given url
#     resp = requests.get(url)
  
#     # saving the xml file
#     with open('.xml', 'wb') as f:
#         f.write(resp.content)

### 1.2 XML Parsing

In [84]:
def parseXML(xmlfile):
    
    # create element tree object
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    
    # find target element tree
    obj_root = tree.find('Obj')
    
    # create data arrays
    author = []
    title = []
    numeric_date = []
    alpha_date = []
    language = []
    source = []
    keywords = []
    
    # iteratre over the trees to extract metadata
    for item in obj_root:
        if item.tag == 'TitleAtt':
            title.append(item[0].text)

        if item.tag == 'Language':
            language.append(item[0].text.strip())

        if item.tag == 'Author':
            title.append(item[0].text)

        if item.tag == 'NumericDate':
            numeric_date.append(item.text)

        if item.tag == 'AlphaDate':
            alpha_date.append(item.text)

        if item.tag == 'Contributors':
            for contributor in item:
                author.append(contributor[0][0].text)

        if item.tag == 'Terms':
            for term in item:
                keywords.append(term[0].text)
        

    return author, title, numeric_date, alpha_date, language, source, keywords

In [85]:
author, title, numeric_date, alpha_date, language, source, keywords = parseXML(xmlfile)

## 2. Sentiment Analysis

### 2.1 spaCY Named Entity Recognition (NER)

### 2.1.1 POS Tags

### 2.1.2 Model